<a href="https://colab.research.google.com/github/tihunn/emotion_testing/blob/main/display_3_datasets_emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import and download database

In [ ]:
import numpy as np                                                    # Массивы
import matplotlib.pyplot as plt                                       # Отрисовка графиков
from google.colab import drive                                        # Связь с google диском
import gdown                                                          # Загрузка из google облака
from tensorflow.keras.utils import to_categorical                     # Преобразование категориальных данных в one hot encoding
import os                                                             # Работа с папками и файлами
import shutil
import time                                                           # Утилиты работы со временем
import random                                                         # Работа со случайными числами
import math                                                           # Математические функции
import pickle                                                         # Сохранение и загрузка структур данных Python
import librosa                                                        # Параметризация аудио
from tensorflow.keras.optimizers import Adam, RMSprop                 # Оптимизаторы для обучения моделей
from tensorflow.keras.models import Sequential, Model, load_model     # Конструирование и загрузка моделей нейронных сетей
from tensorflow.keras.layers import concatenate, Input, Dense, Dropout, BatchNormalization, Flatten, Conv1D, Conv2D, LSTM  # Основные слои
from sklearn.model_selection import train_test_split                  # Разбиение на обучающую и проверочную выборку
from sklearn.preprocessing import LabelEncoder, StandardScaler        # Кодирование категориальных меток, нормирование числовых данных
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay  # Матрица ошибок классификатора
import warnings                                                       # Отключение предупреждений

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# Загружаем kaggle.json (вручную загрузите kaggle.json, который вы скачали, через боковое меню "Файлы" в Colab)
!gdown 1Y5WI645-iPqOS55g4wNKsHaWRAWmCQfi
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Устанавливаем kaggle API, если ещё не установлен
!pip install -q kaggle

# Скачиваем набор данных в текущую директорию
!kaggle datasets download -d ejlok1/toronto-emotional-speech-set-tess -p /content/
!kaggle datasets download -d ejlok1/surrey-audiovisual-expressed-emotion-savee -p /content/
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio -p /content/

# Разархивируем данные, если они находятся в zip формате
!unzip -q /content/toronto-emotional-speech-set-tess.zip
!unzip -q /content/surrey-audiovisual-expressed-emotion-savee.zip
!mkdir -p /content/ravdess/
!unzip -q /content/ravdess-emotional-speech-audio.zip -d /content/ravdess/
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")

print("Path to dataset files:", path)
# Удаляем zip-архивы, если они больше не нужны
!rm -r /content/toronto-emotional-speech-set-tess.zip /content/sample_data \
"/content/tess toronto emotional speech set data" kaggle.json \
/content/surrey-audiovisual-expressed-emotion-savee.zip \
/content/ravdess-emotional-speech-audio.zip


Downloading...
From: https://drive.google.com/uc?id=1Y5WI645-iPqOS55g4wNKsHaWRAWmCQfi
To: /content/kaggle.json
100% 67.0/67.0 [00:00<00:00, 307kB/s]
Dataset URL: https://www.kaggle.com/datasets/ejlok1/toronto-emotional-speech-set-tess
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 99% 422M/428M [00:04<00:00, 90.0MB/s]
100% 428M/428M [00:04<00:00, 108MB/s] 
Dataset URL: https://www.kaggle.com/datasets/ejlok1/surrey-audiovisual-expressed-emotion-savee
License(s): copyright-authors
 99% 106M/107M [00:00<00:00, 153MB/s] 
100% 107M/107M [00:00<00:00, 123MB/s]
Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
 98% 422M/429M [00:02<00:00, 160MB/s]
100% 429M/429M [00:02<00:00, 163MB/s]


100%|██████████| 429M/429M [00:02<00:00, 164MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/uwrfkaggler/ravdess-emotional-speech-audio/versions/1


## упорядочивание папок

In [ ]:
def count_files_in_directory(path, depth=0):
    # Получаем содержимое директории
    try:
        entries = os.listdir(path)
    except FileNotFoundError:
        print("Указанная папка не найдена.")
        return

    # Списки для файлов и поддиректорий
    files = []
    directories = []

    # Разделение файлов и папок в текущей директории
    for entry in entries:
        full_path = os.path.join(path, entry)
        if os.path.isfile(full_path):
            files.append(full_path)
        elif os.path.isdir(full_path):
            directories.append(full_path)

    # Подсчет и вывод форматов файлов
    if files:
        formats = {os.path.splitext(file)[1][1:].lower() for file in files}
        indent = ' ' * (2 * depth)
        print(f"{indent}Папка '{os.path.basename(path)}': {len(files)} файлов, форматы: {', '.join(formats)}")
    else:
        print(f"{' ' * (2 * depth)}Папка '{os.path.basename(path)}' пуста.")

    # Рекурсивно обходим поддиректории с увеличением отступа
    for directory in directories:
        count_files_in_directory(directory, depth + 1)


count_files_in_directory("/content/TESS Toronto emotional speech set data")

Папка 'TESS Toronto emotional speech set data' пуста.
  Папка 'OAF_disgust': 200 файлов, форматы: wav
  Папка 'OAF_angry': 200 файлов, форматы: wav
  Папка 'YAF_neutral': 200 файлов, форматы: wav
  Папка 'YAF_fear': 200 файлов, форматы: wav
  Папка 'OAF_neutral': 200 файлов, форматы: wav
  Папка 'YAF_angry': 200 файлов, форматы: wav
  Папка 'YAF_pleasant_surprised': 200 файлов, форматы: wav
  Папка 'YAF_disgust': 200 файлов, форматы: wav
  Папка 'OAF_Sad': 200 файлов, форматы: wav
  Папка 'YAF_happy': 200 файлов, форматы: wav
  Папка 'YAF_sad': 200 файлов, форматы: wav
  Папка 'OAF_Fear': 200 файлов, форматы: wav
  Папка 'OAF_Pleasant_surprise': 200 файлов, форматы: wav
  Папка 'OAF_happy': 200 файлов, форматы: wav


In [ ]:
def reorganize_folders(base_path):
    # Список для хранения названий созданных папок
    created_folders = []

    # Получаем список всех папок в указанной директории
    for folder_name in os.listdir(base_path):
        # Полный путь к текущей папке
        folder_path = os.path.join(base_path, folder_name)

        # Проверяем, является ли элемент папкой и имеет ли разделитель '_'
        if os.path.isdir(folder_path) and '_' in folder_name:
            # Получаем "последнее слово" после разделителя '_'
            target_folder_name = folder_name.split('_')[-1].lower()
            target_folder_path = os.path.join(base_path, target_folder_name)

            # Обработка специального случая
            if folder_name == 'YAF_pleasant_surprised':
                target_folder_name = 'surprise'
                target_folder_path = os.path.join(base_path, target_folder_name)

            # Создаем папку назначения, если она не существует
            if target_folder_name not in created_folders:
                os.makedirs(target_folder_path, exist_ok=True)
                created_folders.append(target_folder_name)

            # Перемещаем все файлы из текущей папки в папку назначения
            for file_name in os.listdir(folder_path):
                source_file_path = os.path.join(folder_path, file_name)

                # Проверяем, что это файл, а не вложенная папка
                if os.path.isfile(source_file_path):
                    shutil.move(source_file_path, target_folder_path)

            # Удаляем пустую исходную папку
            os.rmdir(folder_path)

    # Возвращаем список созданных папок
    return created_folders

# Пример вызова функции
created_folders = reorganize_folders("/content/TESS Toronto emotional speech set data")
print("Созданные папки:", created_folders)


Созданные папки: ['disgust', 'angry', 'neutral', 'fear', 'surprise', 'sad', 'happy']


In [ ]:
count_files_in_directory("/content/TESS Toronto emotional speech set data")

Папка 'TESS Toronto emotional speech set data' пуста.
  Папка 'happy': 400 файлов, форматы: wav
  Папка 'angry': 400 файлов, форматы: wav
  Папка 'sad': 400 файлов, форматы: wav
  Папка 'neutral': 400 файлов, форматы: wav
  Папка 'surprise': 400 файлов, форматы: wav
  Папка 'disgust': 400 файлов, форматы: wav
  Папка 'fear': 400 файлов, форматы: wav


In [ ]:
import os
import shutil

def distribute_ravdess_files(source_folder, target_folder):
    # Соответствие номеров классов и их названий
    class_map = {
        '01': 'neutral',
        '02': 'neutral',
        '03': 'happy',
        '04': 'sad',
        '05': 'angry',
        '06': 'fear',
        '07': 'disgust',
        '08': 'surprise'
    }

    # Создаем целевые папки для каждого класса, если они еще не созданы
    for class_name in class_map.values():
        os.makedirs(os.path.join(target_folder, class_name), exist_ok=True)

    # Рекурсивно проходим по всем файлам и подкаталогам в исходной папке
    for root, _, files in os.walk(source_folder):
        for file_name in files:
            # Проверяем, является ли файл аудиофайлом в нужном формате
            if file_name.endswith('.wav'):
                # Извлекаем класс из названия файла
                class_number = file_name.split('-')[2]

                # Если класс присутствует в class_map, перемещаем файл
                if class_number in class_map:
                    class_name = class_map[class_number]
                    target_class_folder = os.path.join(target_folder, class_name)

                    # Полные пути к исходному и целевому файлам
                    source_file_path = os.path.join(root, file_name)
                    target_file_path = os.path.join(target_class_folder, file_name)

                    # Перемещаем файл в соответствующую папку
                    shutil.move(source_file_path, target_file_path)
                    print(f"Файл {file_name} перемещен в {target_class_folder}")



distribute_ravdess_files('/content/ravdess', '/content/ravdess1')


Файл 03-01-02-02-01-02-10.wav перемещен в /content/ravdess1/neutral
Файл 03-01-06-01-01-01-10.wav перемещен в /content/ravdess1/fear
Файл 03-01-05-02-01-02-10.wav перемещен в /content/ravdess1/angry
Файл 03-01-02-01-02-02-10.wav перемещен в /content/ravdess1/neutral
Файл 03-01-04-02-01-01-10.wav перемещен в /content/ravdess1/sad
Файл 03-01-04-02-02-01-10.wav перемещен в /content/ravdess1/sad
Файл 03-01-05-02-01-01-10.wav перемещен в /content/ravdess1/angry
Файл 03-01-06-02-01-02-10.wav перемещен в /content/ravdess1/fear
Файл 03-01-03-01-02-01-10.wav перемещен в /content/ravdess1/happy
Файл 03-01-01-01-01-01-10.wav перемещен в /content/ravdess1/neutral
Файл 03-01-06-01-02-01-10.wav перемещен в /content/ravdess1/fear
Файл 03-01-07-01-02-01-10.wav перемещен в /content/ravdess1/disgust
Файл 03-01-03-01-01-01-10.wav перемещен в /content/ravdess1/happy
Файл 03-01-03-02-02-01-10.wav перемещен в /content/ravdess1/happy
Файл 03-01-08-02-02-02-10.wav перемещен в /content/ravdess1/surprise
Файл 0

In [ ]:
count_files_in_directory("/content/TESS Toronto emotional speech set data")

Папка 'TESS Toronto emotional speech set data' пуста.
  Папка 'happy': 400 файлов, форматы: wav
  Папка 'angry': 400 файлов, форматы: wav
  Папка 'sad': 400 файлов, форматы: wav
  Папка 'neutral': 400 файлов, форматы: wav
  Папка 'surprise': 400 файлов, форматы: wav
  Папка 'disgust': 400 файлов, форматы: wav
  Папка 'fear': 400 файлов, форматы: wav


In [ ]:
import os
import shutil

def organize_audio_files(source_folder, dataset_folder='data'):
    # Название папок и соответствие сокращений классов
    class_map = {
        'n': 'neutral',
        'd': 'disgust',
        'sa': 'sad',
        'h': 'happy',
        'su': 'surprise',
        'a': 'angry',
        'f': 'fear'
    }

    # Создаем основную папку набора данных и подкатегории
    os.makedirs(dataset_folder, exist_ok=True)
    for class_name in class_map.values():
        os.makedirs(os.path.join(dataset_folder, class_name), exist_ok=True)

    # Перебираем все файлы в исходной папке
    for file_name in os.listdir(source_folder):
        # Проверяем, что это файл и он имеет формат .wav
        file_path = os.path.join(source_folder, file_name)
        if os.path.isfile(file_path) and file_name.endswith('.wav'):
            # Извлекаем часть после символа "_"
            parts = file_name.split('_')
            if len(parts) < 2:
                continue  # Пропускаем файл, если он не соответствует формату

            # Класс - часть после "_"
            class_code = parts[1][0:2]  # Пробуем извлечь первые две буквы после "_"

            # Проверяем, есть ли такой класс в class_map, иначе проверяем первую букву
            if class_code not in class_map:
                class_code = class_code[0]  # Сокращаем до одной буквы, если двухбуквенный код не найден

            # Если класс найден в class_map, перемещаем файл в нужную папку
            if class_code in class_map:
                target_folder = os.path.join(dataset_folder, class_map[class_code])
                shutil.move(file_path, os.path.join(target_folder, file_name))
                # print(f"Файл {file_name} перемещен в {target_folder}")
            else:
                print(f"Класс не распознан для файла {file_name}")

# Пример вызова функции
organize_audio_files('/content/ALL', "/content/SAVEE")


In [ ]:
count_files_in_directory("/content/SAVEE")

Папка 'SAVEE' пуста.
  Папка 'happy': 60 файлов, форматы: wav
  Папка 'angry': 60 файлов, форматы: wav
  Папка 'sad': 60 файлов, форматы: wav
  Папка 'neutral': 120 файлов, форматы: wav
  Папка 'surprise': 60 файлов, форматы: wav
  Папка 'disgust': 60 файлов, форматы: wav
  Папка 'fear': 60 файлов, форматы: wav


In [ ]:
# prompt: write code for remove half data in /content/SAVEE/neutral

import os
import random

def remove_half_data(directory, num_need):
    """Removes half of the files in a directory at random."""
    try:
        files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
        num_files_to_remove = len(files) - num_need
        files_to_remove = random.sample(files, num_files_to_remove)

        for file in files_to_remove:
            file_path = os.path.join(directory, file)
            os.remove(file_path)
            # print(f"Removed: {file_path}")
        print(f"Removed {num_files_to_remove} files from {directory}")
    except FileNotFoundError:
        print(f"Error: Directory '{directory}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
remove_half_data('/content/SAVEE/neutral', 60)

Removed 60 files from /content/SAVEE/neutral


In [ ]:
# prompt: перемести содержимое каждой папки из /content/SAVEE в соответствующую папку в /content/TESS Toronto emotional speech set data

import shutil
import os

def move_folder_contents(source_dir, destination_dir):
  """Moves the contents of subfolders within a source directory to corresponding subfolders in a destination directory."""
  for folder_name in os.listdir(source_dir):
    source_folder_path = os.path.join(source_dir, folder_name)
    destination_folder_path = os.path.join(destination_dir, folder_name)

    if os.path.isdir(source_folder_path):
      # Create the destination folder if it doesn't exist
      os.makedirs(destination_folder_path, exist_ok=True)

      for item_name in os.listdir(source_folder_path):
        source_item_path = os.path.join(source_folder_path, item_name)
        destination_item_path = os.path.join(destination_folder_path, item_name)

        if os.path.isfile(source_item_path):
          shutil.move(source_item_path, destination_item_path)
          # print(f"Moved '{item_name}' from '{source_folder_path}' to '{destination_folder_path}'")


# Example usage
move_folder_contents("/content/SAVEE", "/content/TESS Toronto emotional speech set data")

In [ ]:
# prompt: перемести все папки и файлы из этой /content/SAVEE папки в эту /content/TESS Toronto emotional speech set data

import shutil

def move_files_and_folders(source_path, destination_path):
    """Moves all files and folders from the source to the destination."""
    try:
        for item in os.listdir(source_path):
            source_item_path = os.path.join(source_path, item)
            destination_item_path = os.path.join(destination_path, item)

            if os.path.isfile(source_item_path):
                shutil.move(source_item_path, destination_item_path)
                print(f"Moved file: {item}")
    except FileNotFoundError:
        print(f"Error: Source path '{source_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
move_files_and_folders("/content/SAVEE", "/content/TESS Toronto emotional speech set data")

In [ ]:
count_files_in_directory("/content/SAVEE")
count_files_in_directory("/content/TESS Toronto emotional speech set data")

Папка 'SAVEE' пуста.
  Папка 'happy' пуста.
  Папка 'angry' пуста.
  Папка 'sad' пуста.
  Папка 'neutral' пуста.
  Папка 'surprise' пуста.
  Папка 'disgust' пуста.
  Папка 'fear' пуста.
Папка 'TESS Toronto emotional speech set data' пуста.
  Папка 'happy': 460 файлов, форматы: wav
  Папка 'angry': 460 файлов, форматы: wav
  Папка 'sad': 460 файлов, форматы: wav
  Папка 'neutral': 460 файлов, форматы: wav
  Папка 'surprise': 460 файлов, форматы: wav
  Папка 'disgust': 460 файлов, форматы: wav
  Папка 'fear': 460 файлов, форматы: wav


In [ ]:
import os
from pathlib import Path
from IPython.display import Audio, display

# Путь к корневой директории с вложенными папками
root_dir = "/content/TESS Toronto emotional speech set data"

# Получаем список вложенных папок
folders = [f for f in Path(root_dir).iterdir() if f.is_dir()]

# Проверяем, что у нас есть хотя бы 7 папок
if len(folders) < 7:
    raise ValueError("Должно быть как минимум 7 папок с аудиофайлами.")

# Проходим по каждой папке и выбираем первый файл формата .wav
for folder in folders[:7]:  # Ограничиваем до первых 7 папок
    wav_files = list(folder.glob("*.wav"))
    if not wav_files:
        print(f"Нет .wav файлов в папке {folder}")
        continue

    # Выбираем первый .wav файл
    audio_file = wav_files[0]
    print(f"Воспроизводится файл: {audio_file}")

    # Воспроизводим аудио
    display(Audio(audio_file))


Воспроизводится файл: /content/TESS Toronto emotional speech set data/happy/YAF_cool_happy.wav


Воспроизводится файл: /content/TESS Toronto emotional speech set data/angry/JK_a07.wav


Воспроизводится файл: /content/TESS Toronto emotional speech set data/sad/YAF_mess_sad.wav


Воспроизводится файл: /content/TESS Toronto emotional speech set data/neutral/OAF_fit_neutral.wav


Воспроизводится файл: /content/TESS Toronto emotional speech set data/surprise/YAF_raid_ps.wav


Воспроизводится файл: /content/TESS Toronto emotional speech set data/disgust/YAF_laud_disgust.wav


Воспроизводится файл: /content/TESS Toronto emotional speech set data/fear/JE_f12.wav
